In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from joblib import Parallel, delayed
import itertools
import xlsxwriter

# Load the dataset from Excel file
excel_file = 'sam9.xlsx'
sheet1 = pd.read_excel(excel_file, sheet_name='Sheet_1')
sheet2 = pd.read_excel(excel_file, sheet_name='Sheet_2')

# Combine the sheets if necessary
data = pd.concat([sheet1, sheet2], ignore_index=True)

# Parse dates and set the index
data['order_date'] = pd.to_datetime(data['order_date'])
data.set_index('order_date', inplace=True)

# Split the data into training and testing sets
train = data[data.index.year < data.index.year.max()]
test = data[data.index.year == data.index.year.max()]

# List of pizzas
pizzas = data['pizza_name'].unique()

# Hyperparameter tuning function
def tune_ets_model(train, pizza_name):
    pizza_data = train[train['pizza_name'] == pizza_name].resample('D').sum().fillna(0)
    quantity = pizza_data['quantity']
    
    # Define the parameter grid
    trend_options = ['add', 'mul', None]
    seasonal_options = ['add', 'mul', None]
    seasonal_periods = [7, 30, 90, 365]
    
    best_mse = float('inf')
    best_params = None
    best_model = None
    
    # Iterate over the parameter grid
    for trend, seasonal, sp in itertools.product(trend_options, seasonal_options, seasonal_periods):
        try:
            model = ExponentialSmoothing(quantity, trend=trend, seasonal=seasonal, seasonal_periods=sp).fit()
            mse = ((model.fittedvalues - quantity) ** 2).mean()
            if mse < best_mse:
                best_mse = mse
                best_params = (trend, seasonal, sp)
                best_model = model
        except:
            continue
    
    return best_model, best_params

# Function to fit the model and forecast
def fit_and_forecast(pizza_name, train, test):
    best_model, best_params = tune_ets_model(train, pizza_name)
    pizza_data = train[train['pizza_name'] == pizza_name].resample('D').sum().fillna(0)
    quantity = pizza_data['quantity']
    
    forecast = best_model.forecast(len(test[test['pizza_name'] == pizza_name].resample('D').sum().fillna(0)))
    
    result = pd.DataFrame({
        'date': test[test['pizza_name'] == pizza_name].resample('D').sum().fillna(0).index,
        'actual': test[test['pizza_name'] == pizza_name].resample('D').sum().fillna(0)['quantity'],
        'forecast': forecast
    }).set_index('date')
    
    return pizza_name, result

# Use parallel processing to fit the model and forecast for each pizza
results = Parallel(n_jobs=-1)(delayed(fit_and_forecast)(pizza, train, test) for pizza in pizzas)

# Initialize lists to store metrics and summary data
summary_data = []
mape_list = []
mae_list = []
mse_list = []
rmse_list = []
accuracy_list = []

# Save the forecasts to an Excel file
with pd.ExcelWriter('forecasts7.xlsx', engine='xlsxwriter') as writer:
    for pizza_name, result in results:
        # Truncate the pizza name to 31 characters for Excel sheet names
        sheet_name = pizza_name[:31]
        result.to_excel(writer, sheet_name=sheet_name)
        
        # Calculate metrics
        mape = np.mean(np.abs((result['actual'] - result['forecast']) / result['actual'])) * 100
        mae = np.mean(np.abs(result['actual'] - result['forecast']))
        mse = np.mean((result['actual'] - result['forecast']) ** 2)
        rmse = np.sqrt(mse)
        accuracy = 100 - mape
        
        # Append metrics to lists
        mape_list.append(mape)
        mae_list.append(mae)
        mse_list.append(mse)
        rmse_list.append(rmse)
        accuracy_list.append(accuracy)

        # Append summary data
        summary_data.append({
            'pizza_name': pizza_name,
            'actual_total': result['actual'].sum(),
            'forecast_total': result['forecast'].sum(),
            'mape': mape,
            'mae': mae,
            'mse': mse,
            'rmse': rmse,
            'accuracy': accuracy
        })

        # Create and save the plot for each pizza
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(result.index, result['actual'], label='Actual')
        ax.plot(result.index, result['forecast'], label='Forecast')
        plt.title(f'Quantity Forecast for {pizza_name}')
        plt.xlabel('Date')
        plt.ylabel('Quantity')
        plt.legend()
        plt.tight_layout()

        # Save the plot as an image
        plot_filename = f'{sheet_name}_plot.png'
        plt.savefig(plot_filename)
        plt.close()

        # Insert the plot into the corresponding sheet
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        worksheet.insert_image('G2', plot_filename)

    # Write summary data to the first sheet
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

    # Create a summary plot
    fig, ax = plt.subplots(figsize=(12, 8))
    summary_df.plot(kind='bar', x='pizza_name', y=['actual_total', 'forecast_total'], ax=ax)
    plt.title('Actual vs Forecasted Total Quantity for Each Pizza')
    plt.xlabel('Pizza Name')
    plt.ylabel('Total Quantity')
    plt.xticks(rotation=90)
    plt.tight_layout()

    # Save the plot as an image
    plt.savefig('summary_plot.png')
    plt.close()

    # Insert the summary plot into the summary sheet
    worksheet = writer.sheets['Summary']
    worksheet.insert_image('I2', 'summary_plot.png')

# Calculate mean metrics
mean_mape = np.mean(mape_list)
mean_mae = np.mean(mae_list)
mean_mse = np.mean(mse_list)
mean_rmse = np.mean(rmse_list)
mean_accuracy = np.mean(accuracy_list)

# Print overall metrics
print(f"Mean MAPE: {mean_mape:.2f}%")
print(f"Mean MAE: {mean_mae:.2f}")
print(f"Mean MSE: {mean_mse:.2f}")
print(f"Mean RMSE: {mean_rmse:.2f}")
print(f"Mean Accuracy: {mean_accuracy:.2f}%")
